In [1]:
import tensorflow as tf
tf.__version__


'2.13.0'

In [2]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.metrics import Precision, Recall
from keras.layers import Flatten
from keras.layers import Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import pandas as pd
from glob import glob
from keras.optimizers import Adam, SGD
import matplotlib.pyplot as plt
import keras

In [3]:
inception = InceptionV3(input_shape = [180,180,3],
    include_top=False,
    pooling='avg',
    weights = 'imagenet')

In [ ]:
inception.summary()

In [4]:
i = 0
for layer in inception.layers:
    layer.trainable = False
    i = i + 1

print(i)

312


In [16]:
img_h,img_w=180,180
batch_size=50
data_dir="Y:\College\Technical Seminar\Iris Dataset\Combined\Clarkson Combined";
test_dir="Y:\\College\\Technical Seminar\\Iris Dataset\\Datasets\\Clarkson 2015\\2015-20180801T100551Z-001\\2015\\Iris\\Clarkson Dataset\\Test"

In [27]:
train_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    label_mode='categorical',
    image_size=(img_h,img_w),
    batch_size=batch_size
)

Found 3021 files belonging to 3 classes.
Using 2417 files for training.


In [28]:
test_ds=tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    seed=123,
    label_mode='categorical',
    image_size=(img_h,img_w),
    batch_size=batch_size
)

Found 1938 files belonging to 3 classes.


In [29]:
val_ds=tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='validation',
    seed=123,
    label_mode='categorical',
    image_size=(img_h,img_w),
    batch_size=batch_size
)

Found 3021 files belonging to 3 classes.
Using 604 files for validation.


In [11]:
out = inception.output
out = Flatten()(out)
out = Dense(512, activation='relu')(out)
out = Dense(512, activation='relu')(out)
prediction = Dense(3, activation='softmax')(out)
model = Model(inputs=inception.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
layers = [(layer.name, layer.trainable) for layer in inception.layers]
df = pd.DataFrame(layers, columns=['Layer Name', 'Layer Trainable'])
pd.set_option('display.max_rows', None)
print(df)

In [30]:
for layer in model.layers[: 249]:
  layer.trainable = False

for layer in model.layers[ 249 :]:
  layer.trainable = True

In [31]:
layers = [(layer.name, layer.trainable) for layer in inception.layers]
df = pd.DataFrame(layers, columns=['Layer Name', 'Layer Trainable'])
pd.set_option('display.max_rows', None)
print(df)

                   Layer Name  Layer Trainable
0                     input_1            False
1                      conv2d            False
2         batch_normalization            False
3                  activation            False
4                    conv2d_1            False
5       batch_normalization_1            False
6                activation_1            False
7                    conv2d_2            False
8       batch_normalization_2            False
9                activation_2            False
10              max_pooling2d            False
11                   conv2d_3            False
12      batch_normalization_3            False
13               activation_3            False
14                   conv2d_4            False
15      batch_normalization_4            False
16               activation_4            False
17            max_pooling2d_1            False
18                   conv2d_8            False
19      batch_normalization_8            False
20           

In [ ]:
layers = [(layer, layer.name, layer.trainable) for layer in inception.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

In [14]:
features=model.layers[-4].output
print(features)
keras.models.Model(inputs=model.input, outputs=features)

KerasTensor(type_spec=TensorSpec(shape=(None, 2048), dtype=tf.float32, name=None), name='flatten/Reshape:0', description="created by layer 'flatten'")


In [23]:
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy',Precision(),Recall()])

In [32]:

callbacks = [
        tf.keras.callbacks.EarlyStopping(patience=4, monitor='val_loss'),
        tf.keras.callbacks.TensorBoard(log_dir='logs')]

history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=15,
  steps_per_epoch=len(train_ds),
  validation_steps=len(val_ds),
  callbacks=callbacks
)

Epoch 1/15
49/49 [==============================] - 147s 3s/step - loss: 0.0407 - accuracy: 0.9859 - precision_1: 0.9859 - recall_1: 0.9847 - val_loss: 0.0438 - val_accuracy: 0.9834 - val_precision_1: 0.9834 - val_recall_1: 0.9834
Epoch 2/15
49/49 [==============================] - 158s 3s/step - loss: 0.0333 - accuracy: 0.9872 - precision_1: 0.9872 - recall_1: 0.9872 - val_loss: 0.0309 - val_accuracy: 0.9901 - val_precision_1: 0.9900 - val_recall_1: 0.9884
Epoch 3/15
49/49 [==============================] - 153s 3s/step - loss: 0.0256 - accuracy: 0.9921 - precision_1: 0.9921 - recall_1: 0.9921 - val_loss: 0.0270 - val_accuracy: 0.9917 - val_precision_1: 0.9917 - val_recall_1: 0.9917
Epoch 4/15
49/49 [==============================] - 148s 3s/step - loss: 0.0409 - accuracy: 0.9868 - precision_1: 0.9868 - recall_1: 0.9868 - val_loss: 0.1250 - val_accuracy: 0.9570 - val_precision_1: 0.9570 - val_recall_1: 0.9570
Epoch 5/15
49/49 [==============================] - 146s 3s/step - loss: 0.0

In [34]:
model.save("Inception_Model_2_clarkson.h5")


In [35]:
test_results = model.evaluate(test_ds, steps=len(test_ds))

print("Test Loss:", test_results[0])
print("Test Accuracy:", test_results[1])
print("Test Precision:", test_results[2])
print("Test Recall:", test_results[3])

39/39 [==============================] - 56s 1s/step - loss: 3.0009 - accuracy: 0.4649 - precision_1: 0.4656 - recall_1: 0.4613
Test Loss: 3.00085711479187
Test Accuracy: 0.4649122953414917
Test Precision: 0.46562498807907104
Test Recall: 0.4613003134727478


In [36]:
predictions = model.predict(test_ds, steps=len(test_ds))

39/39 [==============================] - 60s 1s/step


In [40]:
import numpy as np


predicted_labels = np.argmax(predictions, axis=1)
print(predicted_labels)

[2 2 0 ... 0 0 0]


In [18]:

model2 = tf.keras.models.load_model("Inception_Model_2.2")
model2.compile()

In [19]:
import cv2
import numpy as np
path = "Y:\\College\\Technical Seminar\\Iris Dataset\\Datasets\\IIITD_Contact_Lens_Iris_DB\\Congent Scanner\\Colored\\20120910161836_L.bmp"
# image = cv2.imread(path)
# if image is None:
#     print("Failed to load image")
# else:
#     print("Loaded image successfully")
image=cv2.imread(path)
image_resized=cv2.resize(image,(img_h,img_w))
image=np.expand_dims(image_resized,axis=0)
print(image.shape)

(1, 180, 180, 3)


In [38]:
img_dir = "Y:\\College\\Technical Seminar\\Iris Dataset\\Datasets\\IIITD_Contact_Lens_Iris_DB\\Vista Scanner\\Transparent"

import os
img_files = os.listdir(img_dir)
features_list = []


In [39]:
import os
import numpy as np
import pandas as pd
import cv2
i = 0
for img_file in img_files:
    img_path = os.path.join(img_dir, img_file)
    image=cv2.imread(img_path)
    image_resized=cv2.resize(image,(img_h,img_w))
    image=np.expand_dims(image_resized,axis=0)
    features=model2.layers[-4].output
    feature=keras.models.Model(inputs=model2.input,outputs=features).predict(image)
    features_list.append(feature)
    i = i + 1
    print(i)

1/1 [==============================] - 3s 3s/step
1
1/1 [==============================] - 2s 2s/step
2
1/1 [==============================] - 2s 2s/step
3
1/1 [==============================] - 2s 2s/step
4
1/1 [==============================] - 2s 2s/step
5
1/1 [==============================] - 2s 2s/step
6
1/1 [==============================] - 2s 2s/step
7
1/1 [==============================] - 2s 2s/step
8
1/1 [==============================] - 2s 2s/step
9
1/1 [==============================] - 2s 2s/step
10
1/1 [==============================] - 2s 2s/step
11
1/1 [==============================] - 2s 2s/step
12
1/1 [==============================] - 2s 2s/step
13
1/1 [==============================] - 2s 2s/step
14
1/1 [==============================] - 3s 3s/step
15
1/1 [==============================] - 2s 2s/step
16
1/1 [==============================] - 1s 1s/step
17
1/1 [==============================] - 2s 2s/step
18
1/1 [==============================] - 2s 2s/step
19
1/

In [37]:
features_array = np.vstack(features_list)
df = pd.DataFrame(features_array)
num_features = features_array.shape[1]
col_names = ['feature{}'.format(i) for i in range(1, num_features+1)]
df.columns = col_names
df['filename'] = img_files
df.set_index('filename', inplace=True)
excel_file = "Y:\\College\\Technical Seminar\\InceptionV3 Results\\IIIT Delhi Results\\iiitD_vista_transparent.xlsx"
df.to_excel(excel_file)